In [1]:
from magpi.prelude import *
from magpi import calc

In [2]:
from sklearn import datasets
ds = datasets.fetch_california_housing()

In [3]:
X = array(ds["data"])
y = array(ds["target"])
X.shape, y.shape

((20640, 8), (20640,))

In [12]:
key = random.PRNGKey(53)

class NN(nn.Module):
    @nn.compact
    def __call__(self, x):
        nodes = 15
        activation = nn.gelu
        x = activation(nn.Dense(nodes, name="dense1")(x))
        y = nn.Dense(1, name="dense4")(x)
        return y[0]

model = NN()
key, _key = random.split(key)
init_params = model.init(_key, zeros((8,)))
@jit
def loss(params, X, y):
    return mean((model.apply(params, X) - y) ** 2)

loss(init_params, X, y)

Array(53.703754, dtype=float32)

In [28]:
import math
def dim(params):
    return sum([math.prod(p.shape) for p in tree_leaves(params)])


151

In [13]:
from magpi.sfn import SFN
states = []
Vs = []
Ws = []
eigvals = []
inner_states = []
def cb(step):
    s, eigval, V, W, inner_state = step
    states.append(s.state)
    Vs.append(V)
    Ws.append(W)
    eigvals.append(eigval)
    inner_states.append(inner_state)
solver = SFN(loss, tol=1e-3, k=4, jit=True, maxiter=10, tol_subproblem=1e-4, damping_parameter=1e-2, callback=cb)
res = solver.run(init_params, X, y)

In [14]:
import numpy as np
states[0]

SFNState(iter_num=array(1, dtype=int32), value=array(nan, dtype=float32), grad={'params': {'dense1': {'bias': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan], dtype=float32), 'kernel': array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan]], dtype=float32)}, 'dense4': {'bias': a

In [26]:
inner_states[0]["iter_num"]

array(151, dtype=int32)

In [9]:
res.state.lbfgs_status

Array(0, dtype=int32, weak_type=True)

In [10]:

loss(res.params, X, y), res.state.error

(Array(1.3315504, dtype=float32), Array(2.3297966e-05, dtype=float32))

In [5]:
from jaxopt.loop import while_loop

def _tree_set(M, v, i):
    return tree_map(lambda M, v: M.at[i].set(v), M, v)

def tree_is_zero(t):
    return tree_reduce(lambda a, b: a & jnp.all(b == 0), t, jnp.array(True))

def _for_loop(lower, upper, body_fun, init_val, unroll=False, jit=True):
    
    def cond_fun(state):
        i, _ = state
        return i < upper
        
    def body(state):
        i, state = state
        state = body_fun(i, state)
        return i + 1, state
    
    _, state = while_loop(cond_fun, body, (lower, init_val), maxiter=upper, unroll=unroll, jit=jit)
    return state
        
def lanczos_iteration(k, w0, hvp, last_update=None, unroll=False, jit=True):
    
    alpha = zeros((k,))
    beta = zeros((k,))
    W = tree_map(lambda t: zeros((k, *t.shape)), w0)
    V = tree_map(lambda t: zeros((k, *t.shape)), w0)
    w_last = w0
    v_last = tree_zeros_like(w0)
    
    def body(i, state):
        (V, W, alpha, beta, v_last, w_last) = state
        beta_i = tree_l2_norm(w_last)
        vi = tree_scalar_mul(1 / beta_i, w_last)
        if last_update is None:
            wi = hvp(vi)
        else:
            wi = lax.cond(
                tree_is_zero(last_update),
                lambda: hvp(vi),
                lambda: lax.cond(i == k-1, lambda: last_update, lambda: hvp(vi)
                )
            )
        W = _tree_set(W, wi, i)
        V = _tree_set(V, vi, i)
        
        alpha_i = tree_vdot(wi, vi)
        wi = tree_sub(wi, tree_scalar_mul(alpha_i, vi))
        wi = tree_sub(wi, tree_scalar_mul(beta_i, v_last))
        alpha = alpha.at[i].set(alpha_i)
        beta = beta.at[i].set(beta_i)
        return (V, W, alpha, beta, vi, wi)

    state = (V, W, alpha, beta, v_last, w_last)
    (V, W, alpha, beta, _, _) = _for_loop(0, k, body, state, unroll=unroll, jit=jit)
    return V, W, alpha, beta
        
        
# key = random.PRNGKey(42)
# key, _key = random.split(key)
# # x = random.uniform(_key, (10,))
# last_update = tree_map(zeros_like, x)

H = array([[1.0, 2.0, 3.0], [2.0, 1.0, 2.0], [3.0, 2.0, 1.0]])
#H = random.uniform(random.PRNGKey(53), (5,5))
H = H @ H
# def hvp(p):
#     w = H @ asarray(p)
#     return w
#     return w[0], w[1], w[2]

def f(x):
    x = asarray(x)
    return x @ H @ x

# def df(x):
#     return 2 * H @ asarray(x)

#r0 = tuple(ones((5,)))
r0 = (array(1.0), array(1.2), array(0.5))

# #hvp = lambda p, t: calc.hvp_forward_over_reverse(f, (p,), (t,))
# V, W, alpha, beta = lanczos_iteration(2, r0, hvp, None)#.shape
# V, W, alpha, beta

In [10]:
from magpi.sfn import SFN
solver = SFN(f, tol=1e-6, k=2, jit=True, maxiter=100, tol_subproblem=1e-7)
res = solver.run(r0)

In [11]:
res

OptStep(params=(Array(-6.333655e-08, dtype=float32, weak_type=True), Array(1.2683394e-07, dtype=float32, weak_type=True), Array(-6.333614e-08, dtype=float32, weak_type=True)), state=SFNState(iter_num=Array(39, dtype=int32, weak_type=True), value=Array(1.6005352e-14, dtype=float32), grad=(Array(-5.034674e-07, dtype=float32, weak_type=True), Array(-2.504429e-07, dtype=float32, weak_type=True), Array(-5.034642e-07, dtype=float32, weak_type=True)), error=Array(7.5476964e-07, dtype=float32), last_update=(Array(9.333407e-08, dtype=float32, weak_type=True), Array(-1.8774635e-07, dtype=float32, weak_type=True), Array(9.333463e-08, dtype=float32, weak_type=True)), aux=None, subproblem_converged=Array(True, dtype=bool), subproblem_iter_num=Array(3, dtype=int32, weak_type=True), lbfgs_status=Array(0, dtype=int32, weak_type=True), lbfgs_value=Array(6.4880263e-15, dtype=float32), lbfgs_iter=Array(0, dtype=int32, weak_type=True)))

In [8]:
from jax.scipy.optimize import minimize
from jaxopt._src.tree_util import tree_dot        

def tree_gram(a, b):
    vmap_left = jax.vmap(tree_vdot, in_axes=(0,None))
    vmap_right = jax.vmap(vmap_left, in_axes=(None,0))
    return vmap_right(a, b)

def tree_matmul(a, b):
    _matmul = jax.vmap(tree_vdot, in_axes=(0,None))
    return _matmul(a, b)

x = r0

@partial(jit, static_argnames=("f", ))
def sfn(f, init_params, *args, **kwargs):
    state = init_params
    def body(i, state):
        params = state
        hvp = lambda p: calc.hvp_forward_over_reverse(f, (params,), (p,), *args, **kwargs)
        g = tree_negative(grad(f)(params))
        V, W, alpha, beta = lanczos_iteration(2, g, hvp, None)
        G = tree_gram(V, W)
        lam, v = jnp.linalg.eigh(G)
            
        def fun(l, params, g):
            l = l[0]
            H_inv = v.T * 1 / (jnp.abs(lam) + l) @ v
            alpha = H_inv @ g
            p = tree_add(params, tree_map(lambda v: alpha @ v, V))
            return f(p)

        def inner_body(i, state):
            params, _ = state
            g = grad(f)(params)
            g = -tree_matmul(V, g)
            result = minimize(fun, array([1.0]), (params, g),tol=1e-5, method="BFGS")
            l = result.x[0]
            H_inv = v.T * 1 / (jnp.abs(lam) + l) @ v
            alpha = H_inv @ g
            params_update = tree_map(lambda v: alpha @ v, V)
            params = tree_add(params, params_update)
            return params, params_update
        
        params, params_update = lax.fori_loop(0, 10, inner_body, (params, tree_zeros_like(params)))
        return params
    params = lax.fori_loop(0, 30, body, state)

    return params

In [11]:
params = sfn(f, r0)

In [12]:
params

Array([ 1.4849604e-17, -3.5599283e-17,  1.5445608e-17], dtype=float32)

In [453]:
f(params), f(r0)

(Array(3.6786934e-27, dtype=float32), Array(76.46, dtype=float32))

In [169]:
params

Array([-7.152033 , -6.4241576, -7.6393805], dtype=float32)

In [83]:
fun(1., alpha)

Array(6939.327, dtype=float32)

In [ ]:
v.T * jnp.abs(lam) @ v, G

In [70]:
v.T * jnp.abs(lam) @ v, G

(Array([[5.1206317, 1.4842291],
        [1.484229 , 1.8428023]], dtype=float32),
 Array([[ 4.7918215 ,  2.337143  ],
        [ 2.337143  , -0.36961633]], dtype=float32))

In [45]:
tree_vdot(W, V)

Array(5.9604645e-08, dtype=float32)

In [37]:
V, W, alpha, beta = lanczos_iteration(2, r0, hvp, None)#.shape
V, W, alpha, beta

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


((Array([0.60971075, 0.02822144], dtype=float32),
  Array([ 0.7316529, -0.4044105], dtype=float32),
  Array([0.30485538, 0.91414213], dtype=float32)),
 (Array([2.9875827, 1.9618268], dtype=float32),
  Array([2.560785 , 1.4803166], dtype=float32),
  Array([3.5972934 , 0.18998545], dtype=float32)),
 Array([ 4.7918215 , -0.36961633], dtype=float32),
 Array([1.6401219, 2.337143 ], dtype=float32))

In [28]:
V @ H @ V.T

Array([[ 4.7918215 ,  2.337143  ],
       [ 2.337143  , -0.36961633]], dtype=float32)

In [26]:
H

Array([[1., 2., 3.],
       [2., 1., 2.],
       [3., 2., 1.]], dtype=float32)

In [48]:
last_update = tree_map(zeros_like, x)
df = grad(f)
hvp = lambda p, t: calc.hvp_forward_over_reverse(f, (p,), (t,))
lanczos_iteration(5, df, hvp, ones_like(last_update) * 10, x)#.shape

foo
foo
foo
foo


Array([[ 0.09587097,  1.1853564 ,  0.8064642 ,  1.3451769 ,  1.2022831 ,
         1.2225151 ,  0.91227436,  1.4312901 ,  1.2658458 ,  0.08718801],
       [-0.02851362, -0.35254464, -0.23985583, -0.40007794, -0.35757893,
        -0.36359626, -0.2713255 , -0.4256894 , -0.3764835 , -0.02593116],
       [-0.02851362, -0.35254464, -0.2398558 , -0.4000779 , -0.3575789 ,
        -0.36359626, -0.2713255 , -0.4256894 , -0.37648347, -0.02593116],
       [ 0.02851361,  0.35254458,  0.2398558 ,  0.40007788,  0.3575789 ,
         0.3635962 ,  0.27132547,  0.42568937,  0.37648347,  0.02593116],
       [ 0.06783528,  0.35184446,  0.25307408,  0.39350677,  0.3562569 ,
         0.36153105,  0.28065687,  0.41595492,  0.37282652,  0.06557178]],      dtype=float32)

In [46]:
tree_is_zero(last_update)

Array(True, dtype=bool)

In [36]:
def r(a, b):
    print(a, b)
    return jnp.sum(a) + jnp.sum(b)
tree_reduce(r, last_update, 0.0)

0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Array(0., dtype=float32)

In [38]:
tree_reduce(lambda a, b: a & jnp.all(b == 0), last_update, jnp.array(True))

Array(True, dtype=bool)

In [25]:
import operator
jax.tree.reduce(operator.add, [1, (2, 3), [4, 5, 6]])

21